# Image Denoising Demonstration

This notebook demonstrates various image denoising techniques, with a focus on salt and pepper noise removal.

**Author:** Oussama GUELFAA  
**Date:** 01-04-2025

## 1. Setup and Imports

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, img_as_float
from skimage.util import random_noise

# Add the project root to the Python path
sys.path.append(os.path.dirname(os.path.abspath('')))

# Import the denoising module
from src.image_processing.denoising import (
    add_noise_to_image,
    apply_mean_filter,
    apply_median_filter,
    apply_gaussian_filter,
    apply_bilateral_filter,
    compare_denoising_methods
)
from src.image_processing.denoising.adaptive_median import adaptive_median_filter
from src.image_processing.denoising.noise_generation import load_image

# Configure matplotlib for better display in notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.cmap'] = 'gray'

## 2. Load the Test Image

In [ ]:
# Load the image
image_path = os.path.join("data", "jambe.png")
image = load_image(image_path)

print("Loaded image:", image_path)
print("Image shape:", image.shape)
print("Image min/max values:", image.min(), image.max())

# Display the original image
plt.figure()
plt.imshow(image)
plt.title("Original Image")
plt.axis('off')
plt.show()

## 3. Add Salt and Pepper Noise

In [ ]:
# Add salt and pepper noise
noise_level = 0.1  # 10% of pixels affected
noisy_image = random_noise(image, mode='s&p', amount=noise_level, salt_vs_pepper=0.5)

# Display the noisy image
plt.figure()
plt.imshow(noisy_image)
plt.title("Noisy Image (Salt and Pepper)")
plt.axis('off')
plt.show()

## 4. Apply Different Filters

### 4.1 Maximum Filter

In [ ]:
# Define a function to apply maximum filter
def apply_max_filter(image, kernel_size=3):
    """
    Apply a maximum filter to an image.
    
    Args:
        image (ndarray): Input image
        kernel_size (int): Size of the filter kernel (default: 3)
        
    Returns:
        ndarray: Filtered image
    """
    # Convert to uint8 for rank filters
    from skimage import img_as_ubyte, img_as_float
    from skimage.morphology import disk
    from skimage.filters.rank import maximum
    
    image_uint8 = img_as_ubyte(image)
    
    # Create a disk-shaped structuring element
    selem = disk(kernel_size // 2)
    
    # Apply maximum filter
    filtered_uint8 = maximum(image_uint8, selem)
    filtered = img_as_float(filtered_uint8)
    
    return filtered

# Apply maximum filter
max_filtered = apply_max_filter(noisy_image, kernel_size=5)

# Display the filtered image
plt.figure()
plt.imshow(max_filtered)
plt.title("Maximum Filter")
plt.axis('off')
plt.show()

### 4.2 Minimum Filter

In [ ]:
# Define a function to apply minimum filter
def apply_min_filter(image, kernel_size=3):
    """
    Apply a minimum filter to an image.
    
    Args:
        image (ndarray): Input image
        kernel_size (int): Size of the filter kernel (default: 3)
        
    Returns:
        ndarray: Filtered image
    """
    # Convert to uint8 for rank filters
    from skimage import img_as_ubyte, img_as_float
    from skimage.morphology import disk
    from skimage.filters.rank import minimum
    
    image_uint8 = img_as_ubyte(image)
    
    # Create a disk-shaped structuring element
    selem = disk(kernel_size // 2)
    
    # Apply minimum filter
    filtered_uint8 = minimum(image_uint8, selem)
    filtered = img_as_float(filtered_uint8)
    
    return filtered

# Apply minimum filter
min_filtered = apply_min_filter(noisy_image, kernel_size=5)

# Display the filtered image
plt.figure()
plt.imshow(min_filtered)
plt.title("Minimum Filter")
plt.axis('off')
plt.show()

### 4.3 Mean Filter

In [ ]:
# Apply mean filter
mean_filtered = apply_mean_filter(noisy_image, kernel_size=5)

# Display the filtered image
plt.figure()
plt.imshow(mean_filtered)
plt.title("Mean Filter")
plt.axis('off')
plt.show()

### 4.4 Median Filter

In [ ]:
# Apply median filter
median_filtered = apply_median_filter(noisy_image, kernel_size=5)

# Display the filtered image
plt.figure()
plt.imshow(median_filtered)
plt.title("Median Filter")
plt.axis('off')
plt.show()

### 4.5 Adaptive Median Filter

In [ ]:
# Apply adaptive median filter
adaptive_filtered = adaptive_median_filter(noisy_image, max_window_size=7)

# Display the filtered image
plt.figure()
plt.imshow(adaptive_filtered)
plt.title("Adaptive Median Filter")
plt.axis('off')
plt.show()

## 5. Compare All Filters

In [ ]:
# Create a figure similar to the one in the TP
fig = plt.figure(figsize=(12, 12))
fig.suptitle("Figure 5.7: Different filters applied to the noisy image. The median filter is particularly adapted\nin the case of salt and pepper noise (impulse noise), but still destroy the structures observed in the images.", fontsize=10, y=0.98)

# Create a 3x2 grid for the subplots
gs = fig.add_gridspec(3, 2, hspace=0.2, wspace=0.1)

# Original image
ax1 = fig.add_subplot(gs[0, 0])
ax1.imshow(image)
ax1.set_title('(a) Original image.')
ax1.axis('off')

# Noisy image
ax2 = fig.add_subplot(gs[0, 1])
ax2.imshow(noisy_image)
ax2.set_title('(b) Noisy image (salt and pepper).')
ax2.axis('off')

# Maximum filter
ax3 = fig.add_subplot(gs[1, 0])
ax3.imshow(max_filtered)
ax3.set_title('(c) Maximum filter.')
ax3.axis('off')

# Minimum filter
ax4 = fig.add_subplot(gs[1, 1])
ax4.imshow(min_filtered)
ax4.set_title('(d) Minimum filter.')
ax4.axis('off')

# Mean filter
ax5 = fig.add_subplot(gs[2, 0])
ax5.imshow(mean_filtered)
ax5.set_title('(e) Mean filter.')
ax5.axis('off')

# Median filter
ax6 = fig.add_subplot(gs[2, 1])
ax6.imshow(median_filtered)
ax6.set_title('(f) Median filter.')
ax6.axis('off')

# Adjust layout
plt.subplots_adjust(top=0.92)
plt.show()

# Save the figure
os.makedirs('output', exist_ok=True)
plt.savefig('output/filter_comparison_notebook.png', dpi=300, bbox_inches='tight')

## 6. Quantitative Comparison

Let's compare the performance of different filters using Peak Signal-to-Noise Ratio (PSNR).

In [ ]:
# Define a function to calculate PSNR
def calculate_psnr(original, filtered):
    """Calculate Peak Signal-to-Noise Ratio (PSNR) between two images."""
    mse = np.mean((original - filtered) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 1.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

# Calculate PSNR for each method
psnr_noisy = calculate_psnr(image, noisy_image)
psnr_max = calculate_psnr(image, max_filtered)
psnr_min = calculate_psnr(image, min_filtered)
psnr_mean = calculate_psnr(image, mean_filtered)
psnr_median = calculate_psnr(image, median_filtered)
psnr_adaptive = calculate_psnr(image, adaptive_filtered)

# Create a table of results
results = {
    'Method': ['Noisy Image', 'Maximum Filter', 'Minimum Filter', 'Mean Filter', 'Median Filter', 'Adaptive Median Filter'],
    'PSNR (dB)': [psnr_noisy, psnr_max, psnr_min, psnr_mean, psnr_median, psnr_adaptive]
}

# Display the results
from IPython.display import display
import pandas as pd

df = pd.DataFrame(results)
display(df)

# Plot the results
plt.figure(figsize=(10, 6))
plt.bar(results['Method'], results['PSNR (dB)'])
plt.title('PSNR Comparison of Different Filters')
plt.ylabel('PSNR (dB)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 7. Conclusion

In this notebook, we've demonstrated various filtering techniques for removing salt and pepper noise from images:

1. **Maximum Filter**: Effective for removing 'pepper' noise (dark pixels) but amplifies 'salt' noise (bright pixels).
2. **Minimum Filter**: Effective for removing 'salt' noise but amplifies 'pepper' noise.
3. **Mean Filter**: Reduces random noise but blurs edges and fine details.
4. **Median Filter**: Excellent for salt and pepper noise removal while preserving edges better than the mean filter.
5. **Adaptive Median Filter**: Best overall performance, especially for high noise densities, as it adapts the filter size based on local image characteristics.

The median filter and adaptive median filter are particularly well-suited for salt and pepper noise removal, as they can effectively eliminate impulse noise while preserving important image features.